# Worksheet 08

Name: Calvin Li 
UID: U51621195

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [3]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

s1 = np.random.normal(mean, stdev, 10).tolist()
print(s1)

[5.154814238409788, 4.148564368778786, 5.730261133000761, 4.990661050617047, 6.573232335763221, 5.304612477738084, 3.9961664080998247, 6.592585518114389, 6.418120016456694, 4.8844746481003165]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [4]:
s2 = np.random.normal( 8 , 1 , 10 ).tolist()
print(s2)

[8.078839265952798, 8.045257633790907, 8.96596690000458, 7.949583155946905, 9.027138662140743, 7.526896523406863, 5.2800059012218945, 7.862087146077531, 8.307558086863533, 7.848558967187674]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `s1` and remove it from `s1`, if T then pick the last data point from `s2` and remove it from `s2`. Add these 10 points to a list called `data`.

In [7]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = s1[-1]
        s1 = s1[:-1]
        data.append(p1)
    else:
        p2 = s2[-1]
        s2 = s2[:-1]
        data.append(p2)
print(data)

[7.848558967187674, 8.307558086863533, 4.8844746481003165, 6.418120016456694, 6.592585518114389, 3.9961664080998247, 7.862087146077531, 5.2800059012218945, 7.526896523406863, 5.304612477738084]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

The mean of the two distribution is 5 and 8, and both standard deviation is 1. The probability of them are 0.5 for both. 

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(S_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(S_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [15]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

s1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(s1)
s2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(s2)

prob_s = [ len(s1) / (len(s1) + len(s2)) , len(s2) / len(s1) + len(s2) ]
mean = [ sum(s1)/len(s1) , sum(s2)/len(s2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / len(s1) , sum(map(lambda x : (x - mean[1])**2, s2)) / len(s2) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[7.848558967187674, 8.307558086863533, 6.418120016456694, 6.592585518114389, 7.862087146077531, 7.526896523406863]
[4.8844746481003165, 3.9961664080998247, 5.2800059012218945, 5.304612477738084]
P(S_1) = 0.6,  P(S_2) = 4.666666666666667
mean_1 = 7.425967709684447,  mean_2 = 4.86631485879003
var_1 = 0.47774243371046227,  var_2 = 0.2801832963875638


/Users/calvinli/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


They are far from the true values. 

e) For each data point, compute `P(S_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [16]:
from scipy.stats import norm

prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_s[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_s[0] * pdf_i[0] + prob_s[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_s[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_s[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()


point =  7.848558967187674
probability of observing that point if it came from cluster 0 =  0.5646907149431045
probability of observing that point if it came from cluster 1 =  3.5667467283875585e-25
point =  8.307558086863533
probability of observing that point if it came from cluster 0 =  0.1521530409376325
probability of observing that point if it came from cluster 1 =  2.4930748950824988e-33
point =  4.8844746481003165
probability of observing that point if it came from cluster 0 =  5.975795905543591e-07
probability of observing that point if it came from cluster 1 =  1.4208741801826976
point =  6.418120016456694
probability of observing that point if it came from cluster 0 =  0.09022298716484146
probability of observing that point if it came from cluster 1 =  3.107380718629044e-07
point =  6.592585518114389
probability of observing that point if it came from cluster 0 =  0.1823639341405834
probability of observing that point if it came from cluster 1 =  8.136243262714906e-09
point 

point =  7.848558967187674 from cluster 0 

point =  8.307558086863533 from cluster 0 

point =  4.8844746481003165 from cluster 1

point =  6.418120016456694 from cluster 0

point =  6.592585518114389 from cluster 0

point =  3.9961664080998247 from cluster 1

point =  7.862087146077531 from cluster 0

point =  5.2800059012218945 from cluster 1

point =  7.526896523406863 from cluster 0

point =  5.304612477738084 from cluster 1

f) Having computed `P(S_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(S_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [20]:
prob_c = [sum(prob_s0_x)/ len(prob_s0_x), sum(prob_s1_x) / len(prob_s1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s0_x), sum([x[0] * x[1] for x in zip(prob_s0_x, data)]) / sum(prob_s1_x) ]
var = [ sum(map(lambda x : (x - mean[0])**2, s1)) / sum(prob_s0_x) , sum(map(lambda x : (x - mean[1])**2, s2)) / sum(prob_s1_x) ]

print("P(S_1) = " + str(prob_s[0]) + ",  P(S_2) = " + str(prob_s[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

P(S_1) = 0.6,  P(S_2) = 4.666666666666667
mean_1 = 7.425964161371075,  mean_2 = 11.13892603386055
var_1 = 0.477742780410435,  var_2 = 39.62579111666523


They are completely different from the original values. 

g) Update `P(S_j | X_i)`. Comment on any differences or lack thereof you observe.

In [21]:
prob_s0_x = [] # P(S_0 | X_i)
prob_s1_x = [] # P(S_1 | X_i)
prob_x = [] # P(X_i)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | S_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(S_j) already computed
        prob_c[j]

    # P(X_i) = P(S_0)P(X_i | S_0) + P(S_1)P(X_i | S_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(S_j | X_i) = P(X_i | S_j)P(S_j) / P(X_i)
    prob_s0_x.append( pdf_i[0] * prob_c[0] / prob_x )
    prob_s1_x.append( pdf_i[1] * prob_c[1] / prob_x )

probs = zip(data, prob_s0_x, prob_s1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from S_1 = " + str(p[1]))
    print("Probability of coming from S_2 = " + str(p[2]))
    print()

point =  7.848558967187674
probability of observing that point if it came from cluster 0 =  0.5646869158633452
probability of observing that point if it came from cluster 1 =  0.010033094314248859
point =  8.307558086863533
probability of observing that point if it came from cluster 0 =  0.1521512211640665
probability of observing that point if it came from cluster 1 =  0.010042075417579788
point =  4.8844746481003165
probability of observing that point if it came from cluster 0 =  5.976150419929331e-07
probability of observing that point if it came from cluster 1 =  0.009943112977273733
point =  6.418120016456694
probability of observing that point if it came from cluster 0 =  0.09022462675761035
probability of observing that point if it came from cluster 1 =  0.00999654979020618
point =  6.592585518114389
probability of observing that point if it came from cluster 0 =  0.18236656727368275
probability of observing that point if it came from cluster 1 =  0.01000169770843719
point =  3.

point =  7.848558967187674 from cluster 0 

point =  8.307558086863533 from cluster 0 

point =  4.8844746481003165 from cluster 1

point =  6.418120016456694 from cluster 0

point =  6.592585518114389 from cluster 1

point =  3.9961664080998247 from cluster 0

point =  7.862087146077531 from cluster 0

point =  5.2800059012218945 from cluster 0

point =  7.526896523406863 from cluster 0

point =  5.304612477738084 from cluster 0

h) Use `P(S_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [22]:
hard_assignments = [0 if prob0 > prob1 else 1 for prob0, prob1 in zip(prob_s0_x, prob_s1_x)]
print(hard_assignments)

[0, 0, 1, 0, 0, 1, 0, 1, 0, 1]
